## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jamestown,US,42.10,-79.24,71.71,65,0,6.91,clear sky
1,1,Kapaa,US,22.08,-159.32,84.18,71,20,17.27,few clouds
2,2,Ostrovnoy,RU,68.05,39.51,51.10,84,9,17.13,clear sky
3,3,Rikitea,PF,-23.12,-134.97,72.70,73,63,19.84,broken clouds
4,4,Atuona,PF,-9.80,-139.03,76.98,72,24,24.65,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kapaa,US,22.08,-159.32,84.18,71,20,17.27,few clouds
4,4,Atuona,PF,-9.80,-139.03,76.98,72,24,24.65,few clouds
8,8,Port Blair,IN,11.67,92.75,81.46,81,100,15.57,overcast clouds
25,25,Albany,US,42.60,-73.97,77.41,52,60,1.01,broken clouds
26,26,Georgetown,MY,5.41,100.34,83.89,85,20,0.00,few clouds
29,29,Magnolia,US,33.27,-93.24,79.88,78,76,8.12,broken clouds
35,35,Missoula,US,47.02,-113.80,88.84,22,35,2.86,scattered clouds
39,39,Katsuura,JP,35.13,140.30,83.32,79,13,20.71,few clouds
41,41,Matara,LK,5.95,80.54,76.57,93,93,7.38,overcast clouds
46,46,Vaini,TO,-21.20,-175.20,77.16,78,75,11.50,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                215
City                   215
Country                215
Lat                    215
Lng                    215
Max Temp               215
Humidity               215
Cloudiness             215
Wind Speed             215
Current Description    215
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no empty rows

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,84.18,few clouds,22.08,-159.32,
4,Atuona,PF,76.98,few clouds,-9.80,-139.03,
8,Port Blair,IN,81.46,overcast clouds,11.67,92.75,
25,Albany,US,77.41,broken clouds,42.60,-73.97,
26,Georgetown,MY,83.89,few clouds,5.41,100.34,
29,Magnolia,US,79.88,broken clouds,33.27,-93.24,
35,Missoula,US,88.84,scattered clouds,47.02,-113.80,
39,Katsuura,JP,83.32,few clouds,35.13,140.30,
41,Matara,LK,76.57,overcast clouds,5.95,80.54,
46,Vaini,TO,77.16,broken clouds,-21.20,-175.20,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df_clean = hotel_df.loc[(hotel_df["Hotel Name"]!="")]
hotel_df_clean

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,84.18,few clouds,22.08,-159.32,Sheraton Kauai Coconut Beach Resort Hotel
4,Atuona,PF,76.98,few clouds,-9.80,-139.03,Villa Enata
8,Port Blair,IN,81.46,overcast clouds,11.67,92.75,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
26,Georgetown,MY,83.89,few clouds,5.41,100.34,Cititel Penang
29,Magnolia,US,79.88,broken clouds,33.27,-93.24,Budget Inn
...,...,...,...,...,...,...,...
668,Isangel,VU,75.31,broken clouds,-19.55,169.27,Tanna Lodge
669,Bodden Town,KY,83.93,few clouds,19.28,-81.25,Will T Place
670,Pandan,PH,82.85,overcast clouds,14.05,124.17,Bonifacio Lodge
677,Le Vauclin,MQ,81.84,few clouds,14.55,-60.84,Hôtel Cap Macabou


In [24]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df_clean.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df_clean[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))